In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lmsys-chatbot-arena/sample_submission.csv
/kaggle/input/lmsys-chatbot-arena/train.csv
/kaggle/input/lmsys-chatbot-arena/test.csv


In [2]:
train_file = "/kaggle/input/lmsys-chatbot-arena/train.csv"
test_file = "/kaggle/input/lmsys-chatbot-arena/test.csv"
sample_file = "/kaggle/input/lmsys-chatbot-arena/sample_submission.csv"

train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

sample_df = pd.read_csv(sample_file)

print(train_df.shape, test_df.shape)

(57477, 9) (3, 4)


In [3]:
train_df

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0


In [4]:
test_df, sample_df

(        id                                             prompt  \
 0   136060  ["I have three oranges today, I ate an orange ...   
 1   211333  ["You are a mediator in a heated political deb...   
 2  1233961  ["How to initialize the classification head wh...   
 
                                           response_a  \
 0                    ["You have two oranges today."]   
 1  ["Thank you for sharing the details of the sit...   
 2  ["When you want to initialize the classificati...   
 
                                           response_b  
 0  ["You still have three oranges. Eating an oran...  
 1  ["Mr Reddy and Ms Blue both have valid points ...  
 2  ["To initialize the classification head when p...  ,
         id  winner_model_a  winner_model_b  winner_tie
 0   136060        0.333333        0.333333    0.333333
 1   211333        0.333333        0.333333    0.333333
 2  1233961        0.333333        0.333333    0.333333)

In [5]:
print(f"Total entries in Trainset : {train_df.shape[0]}")
print(f"Unique models used as 'model_a': {train_df['model_a'].nunique()} | Unique models used as 'model_b': {train_df['model_b'].nunique()}")
models_used = pd.DataFrame(train_df['model_a'].value_counts().reset_index().rename(columns={'model_a':'model', 'count': 'count_a'})).merge(pd.DataFrame(train_df['model_b'].value_counts().reset_index().rename(columns={'model_b':'model', 'count':'count_b'})), on=['model'])
models_used

Total entries in Trainset : 57477
Unique models used as 'model_a': 64 | Unique models used as 'model_b': 64


,model,count_a,count_b
0,gpt-4-1106-preview,3678,3709
1,gpt-3.5-turbo-0613,3553,3530
2,gpt-4-0613,3099,3066
3,claude-2.1,2859,2724
4,gpt-4-0314,2087,2035
...,...,...,...
59,falcon-180b-chat,145,141
60,openchat-3.5-0106,108,136
61,qwen1.5-7b-chat,106,102
62,qwen1.5-4b-chat,100,100


In [6]:
f = lambda x: x["model"].split("-")[0]
models_used["base_model"] = models_used.apply(f, axis=1)

print(f"Total unique base model : {models_used['base_model'].nunique()} : \n\t {list(models_used['base_model'].unique())}")
models_used

Total unique base model : 39 : 
	 ['gpt', 'claude', 'vicuna', 'mixtral', 'mistral', 'llama', 'zephyr', 'palm', 'openchat', 'wizardlm', 'koala', 'oasst', 'codellama', 'gemini', 'pplx', 'alpaca', 'yi', 'chatglm', 'RWKV', 'tulu', 'starling', 'qwen', 'chatglm3', 'stripedhyena', 'fastchat', 'openhermes', 'mpt', 'solar', 'deepseek', 'dolly', 'stablelm', 'guanaco', 'llama2', 'chatglm2', 'qwen1.5', 'gpt4all', 'dolphin', 'nous', 'falcon']


,model,count_a,count_b,base_model
0,gpt-4-1106-preview,3678,3709,gpt
1,gpt-3.5-turbo-0613,3553,3530,gpt
2,gpt-4-0613,3099,3066,gpt
3,claude-2.1,2859,2724,claude
4,gpt-4-0314,2087,2035,gpt
...,...,...,...,...
59,falcon-180b-chat,145,141,falcon
60,openchat-3.5-0106,108,136,openchat
61,qwen1.5-7b-chat,106,102,qwen1.5
62,qwen1.5-4b-chat,100,100,qwen1.5
